<a href="https://colab.research.google.com/github/jman4162/aws-for-ml/blob/main/Advanced_Python_Tutorial_on_AWS_Athena_for_Machine_Learning_Scientists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Python Tutorial on AWS Athena for Machine Learning Scientists

Name: John Hodge
Date: 09/23/24

## Table of Contents

1. [Introduction](#introduction)
2. [Prerequisites](#prerequisites)
3. [Setting Up AWS Athena](#setting-up-aws-athena)
4. [Configuring Your Python Environment](#configuring-your-python-environment)
5. [Using Boto3 to Interact with Athena](#using-boto3-to-interact-with-athena)
    - [Establishing a Connection](#establishing-a-connection)
    - [Executing Queries](#executing-queries)
    - [Fetching Query Results](#fetching-query-results)
6. [Using PandasCursor for Simplified Data Retrieval](#using-pandascursor-for-simplified-data-retrieval)
7. [Integrating Athena Queries into Machine Learning Workflows](#integrating-athena-queries-into-machine-learning-workflows)
8. [Optimizing Queries and Performance](#optimizing-queries-and-performance)
9. [Security Best Practices](#security-best-practices)
10. [Advanced Topics](#advanced-topics)
    - [Partitioning and Compression](#partitioning-and-compression)
    - [Using Presto SQL Features](#using-presto-sql-features)
    - [Automating Workflows with AWS Lambda](#automating-workflows-with-aws-lambda)
11. [Conclusion](#conclusion)
12. [References](#references)

---

## Introduction

AWS Athena is a serverless, interactive query service that makes it easy to analyze data in Amazon S3 using standard SQL. For machine learning scientists, Athena provides a powerful tool to preprocess and analyze large datasets without the overhead of managing infrastructure. This tutorial dives deep into leveraging Python, specifically using Boto3 and PandasCursor, to interact with Athena efficiently, integrating it seamlessly into machine learning pipelines.

## Prerequisites

Before diving into the tutorial, ensure you have the following:

- **AWS Account**: Access to AWS services, specifically Athena and S3.
- **IAM Permissions**: Appropriate permissions to use Athena, S3, and other related services.
- **Python Environment**: Python 3.7 or later installed.
- **AWS CLI**: Configured with your credentials.
- **Basic Knowledge**: Familiarity with Python, SQL, and machine learning concepts.

## Setting Up AWS Athena

1. **Create an S3 Bucket for Query Results**:
   
   Athena stores query results in an S3 bucket. If you don’t have one:

   - Navigate to the [S3 Console](https://console.aws.amazon.com/s3/).
   - Click on **Create bucket**.
   - Provide a unique bucket name and select the desired region.
   - Configure settings as needed and create the bucket.

2. **Configure Athena**:

   - Go to the [Athena Console](https://console.aws.amazon.com/athena/).
   - On the first launch, you'll be prompted to set the query result location. Choose the S3 bucket you created.
   
3. **Prepare Your Data**:

   Ensure your data is stored in Amazon S3 in a format supported by Athena (e.g., CSV, JSON, Parquet, ORC).

4. **Define a Database and Tables**:

   Use the Athena console or SQL scripts to define databases and tables that map to your S3 data.

   ```sql
   CREATE DATABASE IF NOT EXISTS ml_database;

   CREATE EXTERNAL TABLE IF NOT EXISTS ml_database.sample_data (
       id INT,
       feature1 DOUBLE,
       feature2 STRING,
       label INT
   )
   ROW FORMAT DELIMITED
   FIELDS TERMINATED BY ','
   STORED AS TEXTFILE
   LOCATION 's3://your-bucket/path/to/data/';
   ```

## Configuring Your Python Environment

1. **Install Required Libraries**:

   ```bash
   pip install boto3 pandas pyathena
   ```

2. **Set Up AWS Credentials**:

   Ensure your AWS credentials are configured. You can set them up using the AWS CLI:

   ```bash
   aws configure
   ```

   Alternatively, set environment variables:

   ```bash
   export AWS_ACCESS_KEY_ID='your_access_key'
   export AWS_SECRET_ACCESS_KEY='your_secret_key'
   export AWS_DEFAULT_REGION='your_region'
   ```

## Using Boto3 to Interact with Athena

Boto3 is the AWS SDK for Python, allowing you to interact with Athena programmatically.

### Establishing a Connection

```python
import boto3
import time

# Initialize the Athena client
athena_client = boto3.client('athena', region_name='your-region')

# Define your S3 output bucket
OUTPUT_BUCKET = 's3://your-output-bucket/path/'
```

### Executing Queries

```python
def execute_query(query, database='ml_database'):
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={'Database': database},
        ResultConfiguration={'OutputLocation': OUTPUT_BUCKET}
    )
    return response['QueryExecutionId']
```

### Fetching Query Results

```python
def get_query_results(query_execution_id):
    # Wait for the query to complete
    while True:
        response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        status = response['QueryExecution']['Status']['State']
        if status in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
            break
        print("Waiting for query to complete...")
        time.sleep(2)
    
    if status == 'SUCCEEDED':
        results = athena_client.get_query_results(QueryExecutionId=query_execution_id)
        return results
    else:
        raise Exception(f"Query {status}")
```

### Complete Example

```python
query = """
SELECT feature1, feature2, label
FROM ml_database.sample_data
WHERE label IS NOT NULL
LIMIT 1000
"""

# Execute the query
query_execution_id = execute_query(query)

# Fetch the results
results = get_query_results(query_execution_id)

# Process results
rows = results['ResultSet']['Rows']
header = [col['VarCharValue'] for col in rows[0]['Data']]
data = [dict(zip(header, [col.get('VarCharValue') for col in row['Data']])) for row in rows[1:]]

import pandas as pd
df = pd.DataFrame(data)
print(df.head())
```

**Note**: While Boto3 provides granular control, handling pagination and data transformation can be cumbersome for large datasets.

## Using PandasCursor for Simplified Data Retrieval

[PandasCursor](https://github.com/laughingman7743/PyAthena) is a convenient interface that allows you to execute Athena queries and fetch results directly into pandas DataFrames.

1. **Install PyAthena**:

   ```bash
   pip install PyAthena[pandas]
   ```

2. **Using PandasCursor**:

   ```python
   from pyathena import connect
   import pandas as pd

   # Establish connection
   conn = connect(
       s3_staging_dir=OUTPUT_BUCKET,
       region_name='your-region',
       aws_access_key_id='your_access_key',
       aws_secret_access_key='your_secret_key'
   )

   # Execute query and fetch into DataFrame
   query = """
   SELECT feature1, feature2, label
   FROM ml_database.sample_data
   WHERE label IS NOT NULL
   LIMIT 1000
   """

   df = pd.read_sql(query, conn)
   print(df.head())
   ```

**Advantages of PandasCursor**:

- **Simplicity**: Directly fetch data into pandas DataFrames.
- **Performance**: Efficient handling of large datasets with support for pagination.
- **Integration**: Seamlessly integrates with pandas-based machine learning workflows.

## Integrating Athena Queries into Machine Learning Workflows

Once data is retrieved from Athena, it can be directly used for machine learning tasks. Here's how you can integrate Athena queries into a typical ML pipeline:

1. **Data Retrieval**:

   Use Boto3 or PandasCursor to fetch the required data.

   ```python
   # Using PyAthena
   df = pd.read_sql(query, conn)
   ```

2. **Data Preprocessing**:

   Clean and preprocess the data as required.

   ```python
   # Handle missing values
   df.fillna(0, inplace=True)

   # Encode categorical variables
   df['feature2'] = df['feature2'].astype('category').cat.codes
   ```

3. **Feature Engineering**:

   Create new features or transform existing ones to improve model performance.

   ```python
   df['feature1_squared'] = df['feature1'] ** 2
   ```

4. **Model Training**:

   Use libraries like scikit-learn, TensorFlow, or PyTorch to train models.

   ```python
   from sklearn.model_selection import train_test_split
   from sklearn.ensemble import RandomForestClassifier

   X = df[['feature1', 'feature2', 'feature1_squared']]
   y = df['label']

   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

   model = RandomForestClassifier(n_estimators=100, random_state=42)
   model.fit(X_train, y_train)

   accuracy = model.score(X_test, y_test)
   print(f"Model Accuracy: {accuracy}")
   ```

5. **Model Evaluation and Deployment**:

   Evaluate model performance and deploy as needed.

## Optimizing Queries and Performance

Efficient querying is crucial when dealing with large datasets. Here are some strategies to optimize Athena queries:

1. **Use Columnar Formats**:

   Formats like Parquet or ORC reduce data size and improve query performance.

   ```sql
   CREATE EXTERNAL TABLE IF NOT EXISTS ml_database.sample_data_parquet (
       id INT,
       feature1 DOUBLE,
       feature2 STRING,
       label INT
   )
   STORED AS PARQUET
   LOCATION 's3://your-bucket/path/to/parquet/';
   ```

2. **Partitioning Data**:

   Partitioning allows Athena to scan only relevant data, reducing query costs and time.

   ```sql
   CREATE EXTERNAL TABLE IF NOT EXISTS ml_database.sample_data_partitioned (
       id INT,
       feature1 DOUBLE,
       feature2 STRING,
       label INT
   )
   PARTITIONED BY (date STRING)
   STORED AS PARQUET
   LOCATION 's3://your-bucket/path/to/partitioned-data/';
   ```

   Add partitions:

   ```sql
   MSCK REPAIR TABLE ml_database.sample_data_partitioned;
   ```

3. **Limit Data Scanned**:

   Use specific columns in your SELECT statement instead of `SELECT *`.

4. **Use Compression**:

   Compress data to reduce storage size and speed up data transfer.

5. **Optimize SQL Queries**:

   - Avoid complex joins if possible.
   - Use WHERE clauses to filter data early.
   - Leverage window functions judiciously.

## Security Best Practices

Ensuring data security and compliance is paramount:

1. **IAM Roles and Policies**:

   - Use least privilege principles.
   - Assign specific permissions to Athena and S3 resources.

2. **Encrypt Data**:

   - Enable server-side encryption for S3 buckets.
   - Use HTTPS for data in transit.

3. **Use Workgroups**:

   Athena Workgroups help manage query usage and apply policies.

4. **Audit and Monitoring**:

   - Enable AWS CloudTrail for logging Athena and S3 activities.
   - Monitor usage with Amazon CloudWatch.

## Advanced Topics

### Partitioning and Compression

Partitioning data by commonly queried columns (e.g., date, region) can significantly enhance query performance. Combined with compression formats like Snappy or GZIP, you can achieve faster data retrieval and lower storage costs.

**Example**:

```sql
CREATE EXTERNAL TABLE IF NOT EXISTS ml_database.sample_data_partitioned (
    id INT,
    feature1 DOUBLE,
    feature2 STRING,
    label INT
)
PARTITIONED BY (year INT, month INT, day INT)
STORED AS PARQUET
LOCATION 's3://your-bucket/path/to/partitioned-data/';
```

### Using Presto SQL Features

Athena is powered by Presto. Leveraging Presto-specific features can enhance your querying capabilities:

- **Window Functions**: Perform complex calculations across rows.
  
  ```sql
  SELECT id, feature1, feature2, label,
         AVG(feature1) OVER (PARTITION BY label) AS avg_feature1
  FROM ml_database.sample_data;
  ```

- **Common Table Expressions (CTEs)**: Simplify complex queries.

  ```sql
  WITH filtered_data AS (
      SELECT * FROM ml_database.sample_data
      WHERE label IS NOT NULL
  )
  SELECT feature1, COUNT(*)
  FROM filtered_data
  GROUP BY feature1;
  ```

### Automating Workflows with AWS Lambda

Automate data retrieval and preprocessing using AWS Lambda:

1. **Create a Lambda Function** that executes Athena queries using Boto3.
2. **Trigger the Function** based on events (e.g., new data arrival in S3).
3. **Integrate with Other AWS Services** for end-to-end automation.

**Example Lambda Function**:

```python
import boto3

def lambda_handler(event, context):
    athena_client = boto3.client('athena', region_name='your-region')
    query = "SELECT * FROM ml_database.sample_data LIMIT 1000"
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={'Database': 'ml_database'},
        ResultConfiguration={'OutputLocation': 's3://your-output-bucket/path/'}
    )
    return {'QueryExecutionId': response['QueryExecutionId']}
```

## Conclusion

AWS Athena, combined with Python's powerful libraries like Boto3 and PandasCursor, offers a robust solution for machine learning scientists to access, preprocess, and analyze large datasets stored in Amazon S3. By following this tutorial, you've learned how to set up Athena, interact with it programmatically, optimize queries, and integrate data retrieval into your machine learning workflows securely and efficiently. Leveraging these tools and best practices will empower you to handle complex data challenges and accelerate your machine learning projects.

## References

- [AWS Athena Documentation](https://docs.aws.amazon.com/athena/)
- [Boto3 Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)
- [PyAthena GitHub Repository](https://github.com/laughingman7743/PyAthena)
- [AWS Security Best Practices](https://aws.amazon.com/architecture/security-best-practices/)
- [Presto SQL Documentation](https://prestodb.io/docs/current/)
